In [1]:
import torch
from typing import List, Union, Tuple
from functools import partial
import json
from transformer_lens.HookedTransformer import HookedTransformer

from algebraic_value_editing.completion_utils import print_n_comparisons
from algebraic_value_editing.prompt_utils import get_x_vector

In [49]:
model_name = "gpt2-xl"
# model_name = "gpt-j-6B"
# model_name = "pythia-2.8b-deduped"

# GPT-J-6B can't load onto GPU RAM of Colab
device: str = "cuda" if (torch.cuda.is_available() and model_name != "gpt-j-6B") else "cpu"
model: HookedTransformer = HookedTransformer.from_pretrained(model_name, device="cpu").to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4                                                                                              │
│   5 # GPT-J-6B can't load onto GPU RAM of Colab                                                  │
│   6 device: str = "cuda" if (torch.cuda.is_available() and model_name != "gpt-j-6B") else "c     │
│ ❱ 7 model: HookedTransformer = HookedTransformer.from_pretrained(model_name, device="cpu").t     │
│   8                                                                                              │
│                                                                                                  │
│ /Users/olejorgensen/Documents/Imperial-2022/Dissertation/activation_additions/add_env/lib/python │
│ 3.10/site-packages/transformer_lens/HookedTransformer.py:880 in from_pretrained                  │
│                                                                                                  │
│    877 │   │                                                                                     │
│    878 │   │   # Get the state dict of the model (ie a mapping of parameter names to tensors),   │
│    879 │   │   # HookedTransformer parameter names.                                              │
│ ❱  880 │   │   state_dict = loading.get_pretrained_state_dict(                                   │
│    881 │   │   │   official_model_name, cfg, hf_model, **from_pretrained_kwargs                  │
│    882 │   │   )                                                                                 │
│    883                                                                                           │
│                                                                                                  │
│ /Users/olejorgensen/Documents/Imperial-2022/Dissertation/activation_additions/add_env/lib/python │
│ 3.10/site-packages/transformer_lens/loading_from_pretrained.py:938 in get_pretrained_state_dict  │
│                                                                                                  │
│    935 │   │   │   │   │   official_model_name, **kwargs                                         │
│    936 │   │   │   │   )                                                                         │
│    937 │   │   │   else:                                                                         │
│ ❱  938 │   │   │   │   hf_model = AutoModelForCausalLM.from_pretrained(                          │
│    939 │   │   │   │   │   official_model_name, **kwargs                                         │
│    940 │   │   │   │   )                                                                         │
│    941                                                                                           │
│                                                                                                  │
│ /Users/olejorgensen/Documents/Imperial-2022/Dissertation/activation_additions/add_env/lib/python │
│ 3.10/site-packages/transformers/models/auto/auto_factory.py:484 in from_pretrained               │
│                                                                                                  │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   483 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 484 │   │   │   return model_class.from_pretrained(                                            │
│   485 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   486 │   │   │   )                                        

## Load Dataset

In [3]:
stories = {}

In [4]:
# Specify the file path
file_path = './datasets/fantasy_200.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

## Run Model (!)!!

In [45]:
from algebraic_value_editing.dataset_utils import ActivationAdditionDataset

activation_addition_dataset = [ActivationAdditionDataset(
    coeff=0.9,
    act_name=0,
    prompt=stories["fantasy"],
    from_dataset=True,
)]

In [46]:
activation_addition_dataset[0].location

'blocks.0.hook_resid_pre'

In [47]:
default_kwargs = {'temperature': 1, 'freq_penalty': 1, 'top_p': .3, 'model': model}
get_x_vector_preset = partial(get_x_vector, pad_method="tokens_left",
                              model=model,
                              custom_pad_id=model.to_single_token(" "))

In [48]:
print_n_comparisons(prompt="Yesterday, my son brought home a pet goose. This was the start of",
                    tokens_to_generate=80, activation_additions=activation_addition_dataset,
                    num_comparisons=8, seed=0, **default_kwargs)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                    Unsteered completions                     |                     Steered completions                      |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my son brought home a pet goose. This was the   |   Yesterday, my son brought home a pet goose. This was the   |
|   start of his life as a dog. He was raised by his mother    |                      start of the day.                       |
| and father in the U.S., and I had to take him out for walks  |                                                              |
| with me every day because he was so small. He has never been |   I got to see it in action and I'm so happy to be able to   |
| shy about taking care of himself, even when he is scared or  | share it with you. It's a big goose tha